In [ ]:
!pip install -U pycld2

In [15]:
import re
import glob
import pandas as pd
import sweetviz
import random
import pandas as pd
import warnings
import unidecode
import numpy as np
import csv
import emoji
from typing import List
from flair.data import Token
import sys,os
import tqdm
import pycld2 as cld2

sys.path.append('../')

In [16]:
import configparser
configParser = configparser.RawConfigParser()   
configFilePath = r'config.txt'
configParser.read(configFilePath)
csvpath = configParser.get('paths', 'csvpath')
outpath = configParser.get('paths', 'outpath')
outpath,csvpath

('./', './data')

In [17]:
from preprocessing.amharicSegmenter import AmharicSegmenter

In [18]:
def find_url(string): 
    text = re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+',string)
    return "".join(text) # converting return value from list to string

In [19]:
def find_eng(string): 
    text = re.findall('[a-zA-Z0-9]',string)
    return "".join(text) # converting return value from list to string

In [20]:
def remove_url(data):
    return data.replace(find_url(data),"")

In [21]:
def remove_eng(data):
    return ' '.join(re.sub(u"[^ሀ-፼]", " ", data).split()).strip()
     

In [22]:
def remove_English_letters(content):
    for idx,cnt in enumerate(content):
        content[idx] = re.sub(r'\s*[A-Za-z]+\b','', cnt)
    return content

In [23]:
def remove_emoji(news):
    news = emoji.demojize(news)
    re.sub('(:\S+)(@\w+)','', news)
    return news

In [24]:
def duplicated_values_data(data):
    dup=[]
    columns=data.columns
    for i in data.columns:
        dup.append(sum(data[i].duplicated()))
    return pd.concat([pd.Series(columns),pd.Series(dup)],axis=1,keys=['Columns','Duplicate count'])

In [25]:
sent_punct = []
word_punct = []
amseg = AmharicSegmenter(sent_punct,word_punct)

# Read each CSV files and write it to text file

In [ ]:
remove = ['\xa087','\xa0', '|', '/','…','«','"','“','”','»','-',
          '$','%','^','&','*','›','’','‘','‹','<','>','`','´','~','=','+']
# remove output file if it exists, then append each csv to a single file output
outfile = outpath+"all_sentences.txt"
os.remove(outfile) if os.path.exists(outfile) else None
# all csv files, to controll progress par
allnews = []
for news in glob.glob(csvpath+"/*.csv"):
    allnews.append(news)
    
# read each files and write to a file system 
with open (outfile,"a", encoding="utf-8") as all_sentences:
    for news in tqdm.tqdm(allnews,  position=0, leave=True):
        data = pd.read_csv(news, names=["ID","URL","Date","Media","Content"] ,encoding="utf-8")
        data = data[data.Content.duplicated()==False].reset_index()
        #data.Content = data.Content.apply(lambda x: remove_url(x))
        #data.Content = data.Content.apply(lambda x: remove_emoji(x))
        data.Content = data.Content.apply(lambda x: remove_eng(x))
        for token in remove:
            data.Content = data.Content.apply(lambda x: x.replace(token,' '))  
        for text in data.Content:
            for s in amseg.tokenize_sentence(text):
                isReliable, textBytesFound, details  = cld2.detect(s)
                if details[0][1] =='am':
                    all_sentences.write(s.replace('\n','')+'\n')



  0%|          | 0/4 [00:00<?, ?it/s]